In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import glob
import nibabel as nb
import itertools
import json
import sys
import statsmodels.api as sm
from statsmodels.formula.api import ols, wls
from statsmodels.formula.api import mixedlm
from patsy.contrasts import Treatment

from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
from statsmodels.discrete.discrete_model import Probit, MNLogit
from statsmodels.tools import add_constant
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFdr
from sklearn.feature_selection import f_regression, mutual_info_regression

import seaborn as sns
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
np.random.seed(1024)

In [2]:
base_dir = '/home/abhijit/Jyotirmay/my_thesis'
basic_feats = ['age', 'sex', 'bmi_numeric']
spleen_sample_cols = ['0_spleen','1_spleen','2_spleen','3_spleen','4_spleen','5_spleen','6_spleen','7_spleen','8_spleen','9_spleen']
liver_sample_cols = ['0_liver','1_liver','2_liver','3_liver','4_liver','5_liver','6_liver','7_liver','8_liver','9_liver']
vols_feat = ['seg_liver', 'seg_spleen']
iou_feats = ['iou_spleen', 'iou_liver', 'iou_mean']
selected_model_feats = basic_feats + spleen_sample_cols + liver_sample_cols + vols_feat + iou_feats + ['volume_id', 'diabetes_status']
selected_dataset_feats = basic_feats +  ['volume_id', 'diabetes_status']
all_paths = [
#     {'full_bayesian': './projects/full_bayesian/reports/full_bayesian_KORA_v2/KORA/10_1571866968.4002764_concat_report_final.csv'},
    {'full_bayesian_0dot01': './projects/full_bayesian/reports/full_bayesian_KORA_v4/KORA/10_1572514598.527084_concat_report_final.csv'},
    {'MC_dropout_quicknat': './projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_KORA_v2/KORA/10_1572006141.7793334_concat_report_final.csv'}, 
    {'probabilistic_quicknat': './projects/probabilistic_quicknat/reports/probabilistic_quicknat_KORA_v2/KORA/10_1571996796.7963011_concat_report_final.csv'}, 
    {'hierarchical_quicknat': './projects/hierarchical_quicknat/reports/hierarchical_quicknat_KORA_v2/KORA/10_1571905560.9377904_concat_report_final.csv'},
    {'dataset_KORA_processed': './dataset_groups/whole_body_datasets/KORA/test_processed_True_concat_report_final.csv'}
]

model_report_paths = {key:val for d in all_paths for key,val in d.items()}

In [42]:
flatten = lambda l: [item for sublist in l for item in sublist]

def transform_to_categorical(df, categorical_features_list):
    for f in categorical_features_list:
        dfDummies = pd.get_dummies(df[f], prefix = f)
        df = pd.concat([df, dfDummies], axis=1)
    return df

def rename(df, cols_map=None):
    if cols_map is None:
        cols_map =  {'bmi-numeric':'bmi_numeric', 'blood-pressure-diastolic':'blood_pressure_diastolic', 'blood-pressure-systolic':'blood_pressure_systolic',
             'cholesterol-hdl':'cholesterol_hdl', 'cholesterol-ldl':'cholesterol_ldl', 'cholesterol-total':'cholesterol_total',
             'mri-liver-fat-artifacts':'mri_liver_fat_artifacts', 'mri-liver-fat-lobus-dexter':'mri_liver_fat_lobus_dexter', 
              'mri-liver-fat-lobus-sinister':'mri_liver_fat_lobus_sinister', 'mri-liver-fat-portal-vein':'mri_liver_fat_portal_vein',
             'meds-lipoprotein-lowering':'meds_lipoprotein_lowering', 'meds-antihypertensive':'meds_antihypertensive',
              'smoker_non-smoker':'smoker_non_smoker','alcohol-g/day':'alcohol_g_day'}
    df.rename(columns=cols_map, inplace=True)
    return df

def z_score_column_normalise(df, column_list):
    normalised_cols_map = {}
    for column in column_list:
        normalised_cols_map[column] = column+'_normalised'
        df[normalised_cols_map[column]] = (df[column] - df[column].mean())/df[column].std(ddof=0)
    return df, normalised_cols_map

def z_score_group_normalise(df, cols_to_normalise):
    normalised_cols_map = {}
    group_cols_value = df[cols_to_normalise].values
    mean, std = np.mean(group_cols_value), np.std(group_cols_value, ddof=0)
    for column in cols_to_normalise:
        normalised_cols_map[column] = 'normalised_'+column
        df[normalised_cols_map[column]] = (df[column] - mean)/std
    return df, normalised_cols_map

def pre_process_for_mlm(df, to_be_transpose_cols, value_name):
    cols = list(df.columns)
    cols_without_tobe_transposed_cols = list(set(cols) - set(to_be_transpose_cols))
    df = df.melt(id_vars=cols_without_tobe_transposed_cols, 
            value_vars=to_be_transpose_cols, 
            value_name=value_name)

    return df

def model_evaluation_matrics(file_paths_dict, cols_to_fetch):
    dicts = {}
    for model, path in file_paths_dict.items():
        if 'dataset' in model:
            continue
        df = pd.read_csv(path)
        dicts[model] = df[cols_to_fetch].iloc[1:].mean()
    return dicts

def individual_feature_stats(feats, df, target_col, categorical_feats=['diabetes_status', 'sex']):
    p_values = {}
    for f in feats:
        try:
            features_string = make_feature_string([f], categorical_feats)
            fii = ols(f'{target_col} ~ {features_string}', df).fit()
            feat_dict = fii.pvalues.to_dict()
            coeffs = fii.params
        
            for k, v in feat_dict.items():
                orig_key = k
                if k == 'Intercept':
                    k = f+'_Intercept'
                p_values[k] = v

                p_values[k+'_coeff'] = coeffs[orig_key]
            p_values['fitting_score'] = fii.rsquared
        except Exception as e:
            print(e)

    return p_values


def make_feature_string(feats, categorical_cols=[]):
    feat_str = '1+'
    for c_col in categorical_cols:
        if type(c_col) is tuple:
            if c_col[0] not in feats:
#                 print(f'{c_col} is not present in given feature list, SKIPPING IT!')
                continue
            feat_str += f'C({c_col[0]}, Treatment(reference={c_col[1]}))+'
            c_col = c_col[0]
        else:
            if c_col not in feats:
#                 print(f'{c_col} is not present in given feature list, SKIPPING IT!')
                continue
            feat_str += f'C({c_col}, Treatment)+'
        feats.remove(c_col)
    
    other_feats_str = '+'.join(feats)
    if other_feats_str is '':
        final_feat_str = feat_str[:-1]
    else:
        final_feat_str = feat_str + other_feats_str

    return final_feat_str
    

def normal_group_fit(df, target_col, features_string):
    model = ols(f'{target_col} ~ {features_string}', df).fit()
    plot_model_outputs(df['bmi_numeric'].values, df[target_col].values, model, 'OLS')
    return model

def weighted_group_feats(df, target_col, features_string, alpha_col):
    alpha = df[alpha_col].values
    model = wls(f'{target_col} ~ {features_string}', df, weights=(1/(1-alpha))).fit()
    return model

def normal_mixed_effect_model(df, target_col, features_string, group_col, L1_wt=None):
    if L1_wt is None:
        model = mixedlm(f'{target_col} ~ {features_string}', df, groups=df[group_col]).fit()
    else:
        model = mixedlm(f'{target_col} ~ {features_string}', df, groups=df[group_col]).fit_regularized(L1_wt=L1_wt)
    return model

def anova_test(ols_model):
    anova_stats = sm.stats.anova_lm(ols_model)
    return anova_stats

def df_from_nested_dicts(dicts):
    df = pd.concat({k+'_'+kk: pd.concat({kk:pd.DataFrame(vv, index=[0]).T}, axis=1) for k, v in dicts.items() for kk, vv in v.items()}, axis=1)
    return df

def df_from_nested_dicts_group(dicts):
    df = pd.concat({k+'_'+kk: pd.concat({kk:pd.DataFrame(vv, index=[0]).T}, axis=1) for k, v in dicts.items() for kk, vv in v.items()}, axis=1)
    return df

def highlight_significance(df, threshold=0.05):
    return df.style.applymap(lambda x: 'background-color : yellow' if x==True or x<threshold else '')


def discrete_group_feature_stats(feats, df, target_col, categorical_feats=['diabetes_status', 'sex'], is_classification=False):
    p_values = {}
    try:
#             features_string = f'1+C({f}, Treatment)' if f in categorical_feats else f'1+{f}'
        x = df[feats]
        y = df[target_col]
        x = add_constant(x)
        model = Probit(y, x)
        fii = model.fit()
#             print(model.pdf(x))
#         fii_ = fii.get_margeff()
#         print(fii.summary())
# #         print(fii_.summary())
        feat_dict = fii.pvalues.to_dict()
        coeffs = fii.params
        
        for k, v in feat_dict.items():
            p_values[k] = v
            p_values[k+'_coeff'] = coeffs[k]

#         for ko, vo in feat_dict.items():
#              for k, v in vo.items():
# #                     print(k, v)
#                 orig_key = k
# #                 if k == 'const':
# #                     k = f+'_Intercept'
#                 p_values[str(ko)+'_'+k] = v
#                 p_values[str(ko)+'_'+k+'_coeff'] = coeffs[ko][orig_key]

        p_values['aic'] = fii.aic
        p_values['bic'] = fii.bic
        p_values['psuedo_r2'] = fii.prsquared
    except Exception as e:
        print('ERROR:', e)

    return p_values, fii

def weighted_group_feats(df, target_col, features_string, alpha_col):
    alpha = df[alpha_col].values
    model = wls(f'{target_col} ~ {features_string}', df, weights=(1/(1-alpha))).fit()
    return model

def discrete_weighted_group_feats(df, target_col, feats, alpha_col):
    alpha = df[alpha_col].values
    x = df[feats]
    y = df[target_col]
    x = add_constant(x)
    model = MNLogit(y, x, weights=(1/(1-alpha))).fit()
    print(model.summary())
    return model

In [31]:
cols_to_fetch = ['sncc', 'ged', 'iou_spleen', 'iou_liver', 'dice_spleen',
       'dice_liver', 'surface_distance_avg_spleen', 'surface_distance_avg_liver']
dicts = model_evaluation_matrics(model_report_paths, cols_to_fetch)
df_model_eval = pd.DataFrame.from_dict(dicts)
df_model_eval = df_model_eval.T
# df_model_eval.columns = cols_to_fetch
df_model_eval

,sncc,ged,iou_spleen,iou_liver,dice_spleen,dice_liver,surface_distance_avg_spleen,surface_distance_avg_liver
full_bayesian_0dot01,0.440058,0.195726,0.778430,0.843838,0.876213,0.921414,0.878335,0.817338
MC_dropout_quicknat,0.400336,0.166126,0.821658,0.878743,0.897251,0.937764,0.915783,0.871014
probabilistic_quicknat,0.164762,0.228213,0.988126,0.992441,0.873734,0.925382,0.875181,0.827286
hierarchical_quicknat,0.188499,0.209206,0.983803,0.989027,0.883031,0.932817,0.897130,0.854920


# Post Process reports

In [32]:
dfs = {}
common_vols = None

for model, path in model_report_paths.items():
    if 'dataset' in model:
        continue
    df = pd.read_csv(path)
    df = rename(df)
    dfs[model] = df
    vols = df[df['iou_mean']>0.51].volume_id.values
    if common_vols is None:
        common_vols = vols
    else:
        common_vols = np.intersect1d(common_vols, vols)
    print(common_vols.shape)

for model, df in dfs.items():
    dfs[model] = df[df.volume_id.isin(common_vols)][selected_model_feats]
    df, normalised_cols = z_score_column_normalise(dfs[model], ['seg_spleen', 'seg_liver'])
    dfs[model] = df
    print('model shape:', dfs[model].shape)
    
df_dataset = pd.read_csv(model_report_paths['dataset_KORA_processed'])
df_dataset = rename(df_dataset)
dfs['dataset_KORA_processed'] = df[df.volume_id.isin(common_vols)][selected_dataset_feats]
print(dfs['dataset_KORA_processed'].shape)

## Change pre-diabetic state to diabetic
for model, df in dfs.items():
    df.loc[df['diabetes_status']==2, 'diabetes_status'] = 1
    dfs[model] = df
    dfs[model].to_csv(f'final_feats_{model}.csv', index=False)

(148,)
(147,)
(147,)
(147,)
model shape: (147, 31)
model shape: (147, 31)
model shape: (147, 31)
model shape: (147, 31)
(147, 5)


# Equations

Eq-1: Diabetes_status = a0 +a1*age + a2*sex + a3*BMI

Eq-2: Diabetes_status = a0 +a1*age + a2*sex + a3*BMI + a4*seg_<spleen|liver>

Eq-3: Diabetes_status = a0 +a1*age + a2*sex + a3*BMI + a4*seg_<spleen|liver> + a5*iou_<spleen|liver>

Eq-4: (Diabetes_status == a0 +a1*age + a2*sex + a3*BMI + a4*seg_<spleen|liver>) * iou_<spleen|liver>

Eq-5: Diabetes_status = a0 +a1*age + a2*sex + a3*BMI +  a4*seg_<spleen|liver>)i

# Group features (eq 1, eq 2 and eq 3)

In [43]:
feats = basic_feats
dicts = {}
anova_test_dicts = {}
for model, df in dfs.items():
#     
#     df, normalised_cols = z_score_column_normalise(df, ['seg_spleen', 'seg_liver'])
#     df = df.fillna(0)

    dicts[model] = {}
    target_col = 'diabetes_status'
#     Equation 1 process
    p_values, statsmodel = discrete_group_feature_stats(basic_feats, df, target_col, ['sex'], True)
    dicts[model][target_col] = p_values
    
    if 'dataset' in model:
        print('dataset cannot be processed!')
        continue

#     Equation 2 process
    feats = basic_feats + [ 'seg_spleen_normalised', 'seg_liver_normalised']
    p_values, statsmodel = discrete_group_feature_stats(feats, df, target_col, ['sex'], True)
    dicts[model][target_col+'_with_seg_volumes'] = p_values
    
#     Equation 2 process
    feats = basic_feats + [ 'seg_spleen_normalised', 'seg_liver_normalised', 'iou_spleen', 'iou_liver']
    p_values, statsmodel = discrete_group_feature_stats(feats, df, target_col, ['sex'], True)
    dicts[model][target_col+'_with_regularised_seg_volumes'] = p_values

p_value_df = df_from_nested_dicts(dicts).T
p_value_df_styler = highlight_significance(p_value_df, 0.05)
p_value_df_styler

Optimization terminated successfully.
         Current function value: 0.466512
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.453328
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.423270
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.466512
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.438315
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.416056
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.466512
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.446703
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.418020
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.466512
  

,,,age,age_coeff,aic,bic,bmi_numeric,bmi_numeric_coeff,const,const_coeff,iou_liver,iou_liver_coeff,iou_spleen,iou_spleen_coeff,psuedo_r2,seg_liver_normalised,seg_liver_normalised_coeff,seg_spleen_normalised,seg_spleen_normalised_coeff,sex,sex_coeff
full_bayesian_0dot01_diabetes_status,diabetes_status,0,0.000138518,0.0516671,145.154,157.116,1.70082e-06,0.124863,1.95267e-10,-7.30687,nan,nan,nan,nan,0.272356,nan,nan,nan,nan,0.0147667,0.638647
full_bayesian_0dot01_diabetes_status_with_seg_volumes,diabetes_status_with_seg_volumes,0,6.42342e-05,0.0622271,145.279,163.221,0.00216086,0.0993853,2.22206e-09,-7.07477,nan,nan,nan,nan,0.292918,0.0553561,0.362874,0.401713,-0.122665,0.123875,0.457745
full_bayesian_0dot01_diabetes_status_with_regularised_seg_volumes,diabetes_status_with_regularised_seg_volumes,0,3.17202e-05,0.0692717,140.441,164.365,0.0161631,0.0881832,0.00735714,-6.39484,0.0508004,-4.35531,0.0130456,3.72449,0.339802,0.0177676,0.512856,0.162197,-0.223586,0.245023,0.367181
MC_dropout_quicknat_diabetes_status,diabetes_status,0,0.000138518,0.0516671,145.154,157.116,1.70082e-06,0.124863,1.95267e-10,-7.30687,nan,nan,nan,nan,0.272356,nan,nan,nan,nan,0.0147667,0.638647
MC_dropout_quicknat_diabetes_status_with_seg_volumes,diabetes_status_with_seg_volumes,0,4.14119e-05,0.0654854,140.865,158.807,0.005537,0.0977853,9.64899e-09,-7.17676,nan,nan,nan,nan,0.316336,0.0128543,0.528675,0.0863165,-0.249205,0.206989,0.395542
MC_dropout_quicknat_diabetes_status_with_regularised_seg_volumes,diabetes_status_with_regularised_seg_volumes,0,5.58475e-05,0.0669754,138.321,162.244,0.00630988,0.102681,0.10392,-6.46544,0.262835,-5.14372,0.0304617,4.30446,0.351054,0.0125101,0.566168,0.0267151,-0.354887,0.214105,0.403354
probabilistic_quicknat_diabetes_status,diabetes_status,0,0.000138518,0.0516671,145.154,157.116,1.70082e-06,0.124863,1.95267e-10,-7.30687,nan,nan,nan,nan,0.272356,nan,nan,nan,nan,0.0147667,0.638647
probabilistic_quicknat_diabetes_status_with_seg_volumes,diabetes_status_with_seg_volumes,0,3.98198e-05,0.0638824,143.331,161.273,0.00468571,0.0917967,7.28572e-09,-6.89797,nan,nan,nan,nan,0.303252,0.372123,-0.128712,0.0199145,0.457968,0.238113,0.360844
probabilistic_quicknat_diabetes_status_with_regularised_seg_volumes,diabetes_status_with_regularised_seg_volumes,0,6.77854e-05,0.0652971,138.898,162.821,0.00532531,0.102897,0.739283,-14.8254,0.0797722,-75.8718,0.0108944,83.7511,0.34799,0.18579,-0.204983,0.014919,0.513348,0.427834,0.254446
hierarchical_quicknat_diabetes_status,diabetes_status,0,0.000138518,0.0516671,145.154,157.116,1.70082e-06,0.124863,1.95267e-10,-7.30687,nan,nan,nan,nan,0.272356,nan,nan,nan,nan,0.0147667,0.638647


# Outputs

A: Base statsmodel comparision with DL statsmodels.

1. (SIMILARITY) Comparision to statsmodel trained on raw dataset, all other statsmodels trained with DL models outputs (segmentation maps), fetures like AGE and BMI show significant relation to DIABETES_STATUS.
2. (DIFFERENCE) Although, SEX has significant relation with DIABETES_STATUS in raw dataset trained statsmodel, but deeplearning outputs based statsmodel does not show the same. 

B: Statsmodel with segmentations output analysis:

1. Full_bayesian does not show any significant relation from segmentations with DIABETES_STATUS
2. MC_dropout and Hierarchical_quicknat shows significant relation from SEG_LIVER_NORMALISED but not from SEG_SPLEEN_NORMALISED with DIABETES_STATUS
3. probabilistic quicknat shows significant relation from SEG_SPLEEN_NORMALISED but nto from SEG_LIVER_NORMALISED with DIABETES_STATUS
4. Considering above result, 2/4 model shows significant relation between SEG_LIVER_NORMALISED (MAJORITY). 

C: Statsmodel with segmentations and iou output analysis.

1. Now with IOU_SPLEEN and IOU_LIVER as inputs features with other feats above, full_bayesian shows significant result from SEG_LIVER_NORMALISED with DIABETES_STATUS
2. With MC_dropout and Hierarchical_quicknat strengthen their significance with DIABETES_STATUS.
3. Probabilistic_quicknat still does not show significance relation between SEG_LIVER_NORMALISED but it strengthen its significance output. 

D: Comparision between statsmodel with and without segmentations and with and without iou_scores.

1. Compare to model without spleen and liver segmentation, model with segmentations are better fit to predict binomial diabetes_status, as per psudo-rsquared values.
2. As per AIC score, all models with segmentation as a features scored less than model without segmentations as feature, except full_bayesian where it missed slightly.
3. With inclusion of IOU_SCORES features, all model shows better fir as per AIC score now, unlike full_bayesian in point 2.
4. Similarly with the inclusion of IOU_SCORES, psuedo_rsquared value improves by 4% wrt statsmodel with segmentation output only and 7-8% comparision to raw statsmodel.


# Regularised Group Feats (eq 4)

In [ ]:
feats = flatten(feats_from_paper_for_group_test_no_categorisation)
feats = ['age', 'sex', 'bmi_numeric']
dicts = {}
anova_test_dicts = {}
for key, value in model_merged_feats_path_combined.items():
    if 'KORA' in key:
#         print('dataset cannot be processed!')
        continue

    df = pd.read_csv(value)
    df = rename(df)
    df = transform_to_categorical(df, ['diabetes_status', 'sex'])
    df, normalised_cols = z_score_column_normalise(df, ['seg_spleen', 'seg_liver'])
    df = df.fillna(0)

    dicts[key] = {}
    anova_test_dicts[key] = {}
    
    target_col = 'diabetes_status'
    best_feats_spleen = feats #choose_best_features(df, feats, target_col)
    feature_string =  make_feature_string(list(best_feats_spleen), [ 'sex'])

    model = discrete_weighted_group_feats(df, target_col, feats , 'iou_spleen')
#     result = anova_test(model)
#     dicts[key][target_col] = model.pvalues.to_dict()
    feat_dict = model.pvalues.to_dict()
    coeffs = model.params
    dicts[key][target_col] = {}
    for ko, vo in feat_dict.items():
             for k, v in vo.items():
#                     print(k, v)
                orig_key = k
#                 if k == 'const':
#                     k = f+'_Intercept'
                dicts[key][target_col][str(ko)+'_'+k] = v
                dicts[key][target_col][str(ko)+'_'+k+'_coeff'] = coeffs[ko][orig_key]

    dicts[key][target_col]['aic'] = model.aic
    dicts[key][target_col]['bic'] = model.bic
    
    feats_ = feats + [ 'seg_spleen_normalised', 'seg_liver_normalised']
    print(feats_)
    model = discrete_weighted_group_feats(df, target_col, feats_ , 'iou_liver')
#     result = anova_test(model)
#     dicts[key][target_col] = model.pvalues.to_dict()
    feat_dict = model.pvalues.to_dict()
    coeffs = model.params
    dicts[key][target_col+'_seg_volumes'] = {}
    for ko, vo in feat_dict.items():
             for k, v in vo.items():
#                     print(k, v)
                orig_key = k
#                 if k == 'const':
#                     k = f+'_Intercept'
                dicts[key][target_col+'_seg_volumes'][str(ko)+'_'+k] = v
                dicts[key][target_col+'_seg_volumes'][str(ko)+'_'+k+'_coeff'] = coeffs[ko][orig_key]

    dicts[key][target_col+'_seg_volumes']['aic'] = model.aic
    dicts[key][target_col+'_seg_volumes']['bic'] = model.bic
#     dicts[key][target_col]['fitting_score'] = model.rsquared
    
#     anova_test_dicts[key][target_col] =  result['PR(>F)'].to_dict()

#     target_col = 'seg_liver_normalised'
#     best_feats_liver = choose_best_features(df, feats, target_col)
#     feature_string =  make_feature_string(list(best_feats_liver), [ 'sex'])
#     model = weighted_group_feats(df, target_col, feature_string , 'iou_liver')
#     result = anova_test(model)
# #     dicts[key][target_col] =  model.pvalues.to_dict()
#     dicts[key][target_col] = {}
#     coeffs = model.params
#     for k, v in model.pvalues.to_dict().items():
#         dicts[key][target_col][k] = v
#         dicts[key][target_col][k+'_coeff'] = coeffs[k]
#     anova_test_dicts[key][target_col] =  result['PR(>F)'].to_dict()

p_value_df = df_from_nested_dicts(dicts).T
# anova_test_p_value_df = df_from_nested_dicts(anova_test_dicts).T
p_value_df_styler = highlight_significance(p_value_df, 0.05)
# anova_test_p_value_df_styler = highlight_significance(anova_test_p_value_df, 0.05)
p_value_df_styler

# Mixed Effect Model (eq 5)

In [ ]:
feats = flatten(feats_from_paper_for_group_test_no_categorisation)
feats = ['age', 'sex', 'bmi_numeric']
dicts = {}
for key, value in model_merged_feats_path_combined.items():
    if 'KORA' in key:
        print('dataset cannot be processed!')
        continue
    df = pd.read_csv(value)
    df = rename(df)
#     df = transform_to_categorical(df, ['diabetes_status', 'sex'])
    df = df.fillna(0)
    df_s = df.copy()
    df_l = df.copy()

    dicts[key] = {}
    
    target_col = 'diabetes_status'
    df_spleen, spleen_normalised_cols_map = z_score_group_normalise(df, spleen_sample_cols)
    df_liver, liver_normalised_cols_map = z_score_group_normalise(df_spleen, liver_sample_cols)
#     df_spleen = pre_process_for_mlm(df_spleen, list(spleen_normalised_cols_map.values()), target_col)
    best_feats_spleen = feats + list(spleen_normalised_cols_map.values()) + list(liver_normalised_cols_map.values()) # choose_best_features(df_spleen, feats, target_col)
    feature_string =  make_feature_string(list(best_feats_spleen), ['sex'])
    p_value_dict_spleen, model = group_feature_stats(feature_string, df_liver, target_col)
    dicts[key][target_col] = p_value_dict_spleen
    dicts[key][target_col]['fitting_score'] = model.rsquared
#     print(key, model.rsquared)
#     df_liver['constant'] = 1
#     v = df_liver[best_feats_spleen+['constant']].values.reshape(-1, 153)
#     fitting_score = model_.score(v)
#     print(key, fitting_score)
        
#     df_liver, liver_normalised_cols_map = z_score_group_normalise(df_l, liver_sample_cols)
# #     df_liver = pre_process_for_mlm(df_liver, list(liver_normalised_cols_map.values()), target_col)
#     best_feats_liver = feats + list(liver_normalised_cols_map.values()) #choose_best_features(df_liver, feats, target_col)
#     feature_string =  make_feature_string(list(best_feats_liver), ['sex'])
#     p_value_dict_liver, model = group_feature_stats(feature_string, df_liver, target_col)
#     dicts[key][target_col+'_vs_liver'] = p_value_dict_liver

p_value_df = df_from_nested_dicts(dicts).T
p_value_df_styler = highlight_significance(p_value_df, 0.05)

p_value_df_styler